<a href="https://colab.research.google.com/github/Anakeyn/complexity2/blob/main/complexity2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Complexity analysis - Practical session 2: NP Completness and the backpack/knapsack problem

In this TP we propose to examine different ways of coding and solving problems of the NP class. We are interested in the problem of the backpack.
We suppose that we have $l$ items each having a utility (or gain) $u_i$. Each of these items has a weight $m_i$. We try to maximize the gain by packing
as many items as possible in a maximum capacity bag $M$. We distinguish two interesting cases :
 - the items are only available in one copy, i.e. we are trying to determine the quantity $x_i \in \{0,1\}$ associated with each item, 
 - one can take several times the same item, i.e. $x_i \in \mathbb{N}^+$. The problem is formalized as follows:
\begin{aligned}
U & =  \text{max}_{x_i} \sum_i x_i u_i \\\
    & \text{s.c.} \sum_i x_i m_i \leq M
\end{aligned}

We will examine different methods to solve this problem allowing you to feel its complexity. You generate for each test that you will make a utility vector and a vector 
of weights that will be randomly drawn integers in $[1.10]$. You will set $M$ according to the number of possible items, for example if you have $n$ items (which will be a 
parameter of your test procedure), you will be able to choose $M=7n$. You will write a function `solve_bag` for each variant that will take the utility, weight and $M$ vectors as parameters and 
will return the maximum value (total gain) reached, as well as the time related to the calculation. 

Write the code below that will generate the possible data for this problem.


In [1]:
#importation des bibliothèques
import numpy as np
import time

In [2]:

#create positions, utilities and weights vectors
def createPositionsUtilitiesWeightsVectors(items = 10) :
  positions  = np.arange(0, items)
  utilities = np.random.randint(low=1, high=10, size=(items))
  wheights = np.random.randint(low=1, high=10, size=(items))
  return positions, utilities, wheights



In [3]:
myPositions, myUtilities, myWeights = createPositionsUtilitiesWeightsVectors(10)

In [4]:
print(myPositions)

[0 1 2 3 4 5 6 7 8 9]


In [5]:
print(myUtilities)

[5 6 1 3 1 3 1 7 8 8]


In [6]:
print(myWeights)

[9 3 9 5 6 6 8 5 7 4]


In [7]:
#Rem : test with itertools to generate all combinations. 
import itertools

stuff = myPositions
tmps1=time.process_time()
for L in range(2, len(stuff) + 1):
    for subset in itertools.combinations(stuff, L):
        print(L)
        print(subset)
tmps2=time.process_time()

2
(0, 1)
2
(0, 2)
2
(0, 3)
2
(0, 4)
2
(0, 5)
2
(0, 6)
2
(0, 7)
2
(0, 8)
2
(0, 9)
2
(1, 2)
2
(1, 3)
2
(1, 4)
2
(1, 5)
2
(1, 6)
2
(1, 7)
2
(1, 8)
2
(1, 9)
2
(2, 3)
2
(2, 4)
2
(2, 5)
2
(2, 6)
2
(2, 7)
2
(2, 8)
2
(2, 9)
2
(3, 4)
2
(3, 5)
2
(3, 6)
2
(3, 7)
2
(3, 8)
2
(3, 9)
2
(4, 5)
2
(4, 6)
2
(4, 7)
2
(4, 8)
2
(4, 9)
2
(5, 6)
2
(5, 7)
2
(5, 8)
2
(5, 9)
2
(6, 7)
2
(6, 8)
2
(6, 9)
2
(7, 8)
2
(7, 9)
2
(8, 9)
3
(0, 1, 2)
3
(0, 1, 3)
3
(0, 1, 4)
3
(0, 1, 5)
3
(0, 1, 6)
3
(0, 1, 7)
3
(0, 1, 8)
3
(0, 1, 9)
3
(0, 2, 3)
3
(0, 2, 4)
3
(0, 2, 5)
3
(0, 2, 6)
3
(0, 2, 7)
3
(0, 2, 8)
3
(0, 2, 9)
3
(0, 3, 4)
3
(0, 3, 5)
3
(0, 3, 6)
3
(0, 3, 7)
3
(0, 3, 8)
3
(0, 3, 9)
3
(0, 4, 5)
3
(0, 4, 6)
3
(0, 4, 7)
3
(0, 4, 8)
3
(0, 4, 9)
3
(0, 5, 6)
3
(0, 5, 7)
3
(0, 5, 8)
3
(0, 5, 9)
3
(0, 6, 7)
3
(0, 6, 8)
3
(0, 6, 9)
3
(0, 7, 8)
3
(0, 7, 9)
3
(0, 8, 9)
3
(1, 2, 3)
3
(1, 2, 4)
3
(1, 2, 5)
3
(1, 2, 6)
3
(1, 2, 7)
3
(1, 2, 8)
3
(1, 2, 9)
3
(1, 3, 4)
3
(1, 3, 5)
3
(1, 3, 6)
3
(1, 3, 7)
3
(1, 3, 8)
3
(1, 3, 9)
3
(1, 4

In [8]:
print('Execution time={}'.format((tmps2-tmps1)))

Execution time=0.5109358110000004


## Brute force approach

We don't bother with complex considerations here: write a method that calculates all possible combinations ($2^n$!), evaluates them, and returns the optimal gain. 


In [74]:
#fonction qui retourne la meilleure combinaison et le top gain en testant toutes els combinaisons possibles sans remises

def BestCombinationTopGainBruteForce(myPositions, myUtilities, myWeights, n) :
  topGain = 0
  topListCombination = []
  M = 7*n #Max Weihghts
  for L in range(2, len(myPositions) + 1):
      for tupSubset in itertools.combinations(myPositions, L):
        #print("Tuple",tupSubset)
        #print(type(tupSubset))
        listSubset = list(tupSubset)  #Tuple in list
        if (sum(myWeights[listSubset]) < M) :
            #print("sum myWeights < M=", M, " ",sum(myWeights[listSubset]))
            if (sum(myUtilities[listSubset]) > topGain) :
              #print("sum myUtilitiesSubset > topGain", sum(myUtilities[listSubset])) 
              topGain = sum(myUtilities[listSubset])
              topListCombination = listSubset

  return topListCombination, topGain

In [93]:
#execution
n=10
myPositions, myUtilities, myWeights = createPositionsUtilitiesWeightsVectors(n)
tmps1=time.process_time()
myBestCombination, myTopGain = BestCombinationTopGainBruteForce(myPositions, myUtilities, myWeights, n)
tmps2=time.process_time()


In [94]:
 #temps d'execution
print('Execution time={}'.format((tmps2-tmps1)))

Execution time=0.011221487000057095


In [95]:
#Entrées
print("Max Poids = ", n*7)
print("myPositions",myPositions)
print("myUtilities",myUtilities)
print("myWeights",myWeights)
#Resultats :
print("myTopGain ", myTopGain)
print("myBestCombination ",myBestCombination)

Max Poids =  70
myPositions [0 1 2 3 4 5 6 7 8 9]
myUtilities [3 8 6 7 5 4 7 4 4 2]
myWeights [5 7 8 1 1 1 3 2 7 1]
myTopGain  50
myBestCombination  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


Same thing but this time you can choose the same item several times. For each item we can determine the maximum limit of the number of times we can choose this item as the integer part of 
$M/m_i$. Be careful, calculation times can become very long for high $M$ values.

In [178]:
#fonction qui retourne la meilleure combinaison et le top gain en testant toutes les combinaisons possibles avec remise. Pour la remise limitée voir plus bas

def BestCombinationTopGainBruteForceWithReplacement(myPositions, myUtilities, myWeights, n) :
  topGain = 0
  topListCombination = []
  M = 7*n #Max Weights
  for L in range(2, len(myPositions) + 1):
      for tupSubset in itertools.combinations_with_replacement(myPositions, L):
        #print("Tuple",tupSubset)
        #print(type(tupSubset))
        listSubset = list(tupSubset)  #Tuple in list
        if (sum(myWeights[listSubset]) < M) :
            #print("sum myWeights < M=", M, " ",sum(myWeights[listSubset]))
            if (sum(myUtilities[listSubset]) > topGain) :
              #print("sum myUtilitiesSubset > topGain", sum(myUtilities[listSubset])) 
              topGain = sum(myUtilities[listSubset])
              topListCombination = listSubset

  return topListCombination, topGain

In [135]:
tmps1=time.process_time()
myBestCombinationWR, myTopGainWR = BestCombinationTopGainBruteForceWithReplacement(myPositions, myUtilities, myWeights, n)
tmps2=time.process_time()

In [136]:
 #temps d'execution
print('Execution time={}'.format((tmps2-tmps1)))

Execution time=1.3819438729999547


In [177]:
#Entrées
print("Max Poids = ", n*7)
print("myPositions",myPositions)
print("myUtilities",myUtilities)
print("myWeights",myWeights)
#Resultats :
print("myTopGainWR ", myTopGainWR)
print("myBestCombinationWR ",myBestCombinationWR)

Max Poids =  70
myPositions [0 1 2 3 4 5 6 7 8 9]
myUtilities [3 8 6 7 5 4 7 4 4 2]
myWeights [5 7 8 1 1 1 3 2 7 1]
myTopGainWR  79
myBestCombinationWR  [1, 1, 1, 1, 1, 1, 1, 1, 1, 3]


In [180]:
#Limitation du nombre d'items inspiré de itertools
def combinations_with_max_replacement(myPositions, myUtilities, myWeights, r) :
    # combinations_with_replacement('ABC', 2) --> AA AB AC BB BC CC
    pool = tuple(myPositions)
    n = len(pool)
    M = 7*n #Max Weights
    limitItems = r
    if not n and r:
        return
    indices = [0] * r
    #print("indices", indices)
    yield tuple(pool[i] for i in indices)

    while True:
        
        #for i in reversed(range(r)):
        for i in reversed(range(limitItems)):
            if indices[i] != n - 1:
                break
        else:
            return
        
        limitItems = min(r,int(M/myWeights[indices[i]]))  #ici calcul du max de remplacements
        print("limitItems=",limitItems)
        #on force 
        #limitItems = 2
        #indices[i:] = [indices[i] + 1] * (r - i)
        indices[i:] = [indices[i] + 1] * (limitItems - i)
        yield tuple(pool[i] for i in indices)

In [181]:
#fonction qui retourne la meilleure combinaison et le top gain en testant toutes les combinaisons possibles avec remise limitée par int(M/mi)

def BestCombinationTopGainBruteForceWithReplacementLimited(myPositions, myUtilities, myWeights, n) :
  topGain = 0
  topListCombination = []
  M = 7*n #Max Weights
  for L in range(2, len(myPositions) + 1):
      for tupSubset in combinations_with_max_replacement(myPositions, myUtilities, myWeights, L):
        #print("Tuple",tupSubset)
        #print(type(tupSubset))
        listSubset = list(tupSubset)  #Tuple in list
        if (sum(myWeights[listSubset]) < M) :
            #print("sum myWeights < M=", M, " ",sum(myWeights[listSubset]))
            if (sum(myUtilities[listSubset]) > topGain) :
              #print("sum myUtilitiesSubset > topGain", sum(myUtilities[listSubset])) 
              topGain = sum(myUtilities[listSubset])
              topListCombination = listSubset

  return topListCombination, topGain

In [182]:
tmps1=time.process_time()
myBestCombinationWRL, myTopGainWRL = BestCombinationTopGainBruteForceWithReplacementLimited(myPositions, myUtilities, myWeights, n)
tmps2=time.process_time()

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
limitItems= 8
limitItems= 8
limitItems= 8
limitItems= 8
limitItems= 8
limitItems= 8
limitItems= 8
limitItems= 8
limitItems= 8
limitItems= 8
limitItems= 8
limitItems= 8
limitItems= 8
limitItems= 8
limitItems= 8
limitItems= 8
limitItems= 8
limitItems= 8
limitItems= 8
limitItems= 8
limitItems= 8
limitItems= 8
limitItems= 8
limitItems= 8
limitItems= 8
limitItems= 8
limitItems= 8
limitItems= 8
limitItems= 8
limitItems= 8
limitItems= 8
limitItems= 8
limitItems= 8
limitItems= 8
limitItems= 8
limitItems= 8
limitItems= 8
limitItems= 8
limitItems= 8
limitItems= 8
limitItems= 8
limitItems= 8
limitItems= 8
limitItems= 8
limitItems= 8
limitItems= 8
limitItems= 8
limitItems= 8
limitItems= 8
limitItems= 8
limitItems= 8
limitItems= 8
limitItems= 8
limitItems= 8
limitItems= 8
limitItems= 8
limitItems= 8
limitItems= 8
limitItems= 8
limitItems= 8
limitItems= 8
limitItems= 8
limitItems= 8
limitItems= 8
limitItems= 8
limitItems= 

KeyboardInterrupt: ignored

In [175]:
 #temps d'execution
print('Execution time={}'.format((tmps2-tmps1)))

Execution time=2.646689705999961


In [176]:
#Entrées
print("Max Poids = ", n*7)
print("myPositions",myPositions)
print("myUtilities",myUtilities)
print("myWeights",myWeights)
#Resultats :
print("myTopGainWRL ", myTopGainWRL)
print("myBestCombinationWRL ",myBestCombinationWRL)

Max Poids =  70
myPositions [0 1 2 3 4 5 6 7 8 9]
myUtilities [3 8 6 7 5 4 7 4 4 2]
myWeights [5 7 8 1 1 1 3 2 7 1]
myTopGainWRL  77
myBestCombinationWRL  [1, 1, 1, 1, 1, 1, 1, 6, 6, 6]


## Greedy approach

For each object the gain/mass ratio ($u_i/m_i$) is calculated. The objects are sorted in descending order, then the bag is filled in this order until no more items can be added.
Compare the quality of the solution obtained with that of the previous exact solver. In particular, find cases where the gluttonous strategy does not give the optimal solution of the 
problem. Here again you will code two versions of the function (only one item available and unlimited number of items available). 


## Dynamic programming

We limit ourselves here to the case where only one item of each object is available.

The idea of dynamic programming is to solve incrementally simpler versions of the problem, and to store intermediate results necessary to 
add new variables. A time-space compromise is then made. In the case of the backpack problem, the problem is said to be with *optimal substructure*, that is to say that we 
can find the optimal value of the problem at variable $i$ from the optimal value at variable $i-1$. 
The following quantity $P(k,m)$, describing the state of the system for k variables, is defined by recurrence :
$$
\begin{eqnarray}
P(k,m) & = & \text{max}_{x_i} \sum^k_i x_i u_i \\
    &    & \text{s.c.} \sum_i x_i m_i \leq m
\end{eqnarray}
$$
then the optimal solution is either :
 - the optimal solution $P(k-1,m)$ where one chooses not to add the item, i.e. $x_k=0$.
- the optimal solution $P(k-1,m-m_k)+u_k$ where we choose to add the item, i.e. $x_k=1$.

You just have to build a table of the different possibilities $P(k,m)$. Once this table is built, you just have to start from the element 
$P(k,M)$ and go back to the $P(0,.)$ element to find out whether you choose the item or not and thus construct the solution.

We note then that the complexity of the algorithm is in time and space $o(nM)$. Although polynomial, we have not shown that $P=NP$: 
since the coding of $M$ is done on $log(M)$ bits, we remain within the exponential complexity of the size of the input.
  

Code and test this algorithm.

Optionnaly, you can get some help from the Wikipedia page on [0-1 knapasack problem](https://en.wikipedia.org/wiki/Knapsack_problem).

In [188]:
#Dynamic approach with one item only in each backpack
#note M is Max Weight in the bag
def DynamicSolveBagOneItem(M, utility, weights) :
  print("Max Weights", M)

  bestSequence = ""
  cumulSequence = ""
  bestGain = 0
  currentGain = 0
  currentWeight = 0
  cumulGain = 0
  cumulWeight = 0
  i=0  #on démarre au premier item
  
  while (i<len(utility) ) :  #first item
      print("i", i)
      cumulSequence = str(i)
      cumulGain = 0
      cumulWeight = 0
      currentGain = utility[i]
      currentWeight = weights[i]
      if currentWeight > M :  #vérifie si le premier item n'est pas superieur au max de poids
        print("currentWeight FIRST item", currentWeight)
        i=i+1
      else :
        cumulGain = currentGain 
        cumulWeight = currentWeight
        j=0  
        while (j < len(utility) ) : #next item
          print("j", j)
          print("1 currentWeight + cumulWeight ", currentWeight+cumulWeight) 
          if (i==j) : j=i+1 #on ne prend pas le premier
          if (j>= len(utility)) : break
          currentGain = utility[j]
          currentWeight = weights[j]
          if currentWeight > M :  #vérifie si l'item suivant n'est pas superieur au max de poids on ne le prend pas
            print("currentWeight next item", currentWeight)
            j=j+1  #on va tester le suivant
          else :
            if currentWeight+cumulWeight > M : #vérifie si le cumul n'est pas superieur au max de poids on ne le prend pas
                print("2 currentWeight + cumulWeight ", currentWeight+cumulWeight)  
                j=j+1  # on ne prend pas l'objet et on va chercher le suivant 
            else : #on prend l'item
                cumulWeight = currentWeight+cumulWeight
                cumulGain = currentGain+cumulGain
                cumulSequence = cumulSequence + str(j)
                if cumulGain > bestGain :
                    bestSequence = cumulSequence
                    bestGain = cumulGain
                    print("cumulWeight ", cumulWeight)
                    print("bestGain ", bestGain)
                    print("bestSequence ", bestSequence)
                j=j+1
      i=i+1    
  return(bestGain)






In [189]:
#appel à la fonction
DynamicSolveBagOneItem(40, myUtility, myWeights)

NameError: ignored

## Summary and final conclusions
For each method, vary the number of items $n$, and measure an average time taken on $10$ resolution of the problem. Draw the corresponding average execution time curves
for the 3 methods in both cases (one or more times the same item, where dynamic programming will be excluded). 